## New Search using LLAMA 3.1

#### Detailed Approach to bring the search together
<b> <i> Keyword Combinations
1. Company Name + Leadership + COO + CEO + CXO
2. Company Name + Merger + Acquisition
3. Company Name + Legal +Cases + Verdict
4. Company Name + Latest News
5. Company Name : News API
    
<b><i> API Resources
1. SerpAPI

<b> <i> Each function should provide all the tags and keywords
    
1. API Function + Keyword Combination --> Output

2. Collate unique links
    
3. Ensure full/partial company match is in the tag descriptions
    
4. For final set of links and similar news , how to get 
    
5. Use a SEO API to validate the ranking of the website Ahrefs, mozapi, SEM Rush , Majestic API. Order of the link in the SERPAPI also points towards it trustworthyness


    

### 1. Search and append to a base json

In [1]:
import yaml
import re
from api_search import *
import json

In [210]:
# with open("config.yml", "w") as file:
#     yaml.dump(dt, file, default_flow_style=False, sort_keys=False)

with open("config.yml", "r") as file:
    api_keys = yaml.safe_load(file)

# print(api_keys)

In [3]:
query = "f w webb"
c1 = "latest news"
c2 = "merger acquisition"
c3 = "court legal issues"
c4 = "leadership appointment"

In [4]:
# out_qc1 = serp_google_search(query=f"{query} {c1}", serp_key = api_keys['serp_api_key'], nws=False, top_res = 50)
# out_qc2 = serp_google_search(query=f"{query} {c2}", serp_key = api_keys['serp_api_key'], nws=False, top_res = 50)
# out_qc3 = serp_google_search(query=f"{query} {c3}", serp_key = api_keys['serp_api_key'], nws=False, top_res = 50)
# out_qc4 = serp_google_search(query=f"{query} {c4}", serp_key = api_keys['serp_api_key'], nws=False, top_res = 50)
# out_qnews = serp_google_search(query=f"{query}", serp_key = api_keys['serp_api_key'], nws=True, top_res = 50)

In [5]:
file_path = "results.jsonl"

In [6]:
# append_to_json(file_path, out_qc1)
# append_to_json(file_path, out_qc2)
# append_to_json(file_path, out_qc3)
# append_to_json(file_path, out_qc4)
# append_to_json(file_path, out_qnews)

### 2. Load the json to parse

In [7]:
# Example usage
data = read_json_as_dicts(file_path)
print(len(data))

5


### 3. Take the specific search term as the context
### Use LLAMA 3.1 to cleanup the results :

For each response there are related questions, links and answers 
Further there are ranked order list of information.
Prioritize information with a date element to it, skip it otherwise
Maintain the order of the information.


In [8]:
data[1]['search_information']['query_displayed']

'f w webb merger acquisition'

In [9]:
data[1]

{'search_metadata': {'id': '67457e3220e6ebfd3a6dd9bc',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/9ff241d797233871/67457e3220e6ebfd3a6dd9bc.json',
  'created_at': '2024-11-26 07:52:18 UTC',
  'processed_at': '2024-11-26 07:52:18 UTC',
  'google_url': 'https://www.google.com/search?q=f+w+webb+merger+acquisition&oq=f+w+webb+merger+acquisition&uule=w+CAIQICIaQXVzdGluLFRleGFzLFVuaXRlZCBTdGF0ZXM&hl=en&gl=us&num=50&sourceid=chrome&ie=UTF-8',
  'raw_html_file': 'https://serpapi.com/searches/9ff241d797233871/67457e3220e6ebfd3a6dd9bc.html',
  'total_time_taken': 0.52},
 'search_parameters': {'engine': 'google',
  'q': 'f w webb merger acquisition',
  'location_requested': 'Austin, Texas, United States',
  'location_used': 'Austin,Texas,United States',
  'google_domain': 'google.com',
  'hl': 'en',
  'gl': 'us',
  'num': '50',
  'device': 'desktop'},
 'search_information': {'query_displayed': 'f w webb merger acquisition',
  'total_results': 98000,
  'time_taken_disp

1. Apply the name filter and date filter to ensure we only pick the snippets with company name and with a data
2. Title and snippets are joined together to have more context


In [10]:
org_dict = {z['position']: '.'.join([re.subn('\W+', ' ',z['title'])[0].lower(), re.subn('\W+', ' ',z['snippet'])[0].lower()]) for z in data[1]['organic_results'] if 'date' in z.keys()}
org_dict = {x:y for x,y in org_dict.items() if query in y}

In [11]:
org_dict[10]

'f w webb co acquires stritt priebe and vj stanley.f w webb company announced the acquisition of stritt priebe a specialist in industrial valve supply service and repair in buffalo '

### 4. Use Llama to further filter

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

from langchain.schema.runnable import RunnableLambda, RunnableSequence

from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain.schema.output_parser import StrOutputParser

from langchain_community.chat_message_histories import SQLChatMessageHistory

from langchain_ollama.llms import OllamaLLM
from langchain_ollama import ChatOllama

# Keeping very low temprature to maintain replicability

llm = ChatOllama(model='llama3.1', temperature=0.1)
model = OllamaLLM(model="llama3.1", temperature=0.1, num_predict=100)

llm32 = ChatOllama(model='llama3.2', temperature=0.1)
model32 = OllamaLLM(model="llama3.2", temperature=0.5, num_predict=100)

llm321b = ChatOllama(model='llama3.2:1b', temperature=0.1)
model321b = OllamaLLM(model="llama3.2:1b", temperature=0.5, num_predict=100)

llm_phi = ChatOllama(model='phi3:14b', temperature=0.1)
model_phi = OllamaLLM(model="phi3:14b", temperature=0.1, num_predict=100)

llm_qa = ChatOllama(model='llama3-chatqa:latest', temperature=0.1)
model_qa = OllamaLLM(model="llama3-chatqa:latest", temperature=0.1, num_predict=100)


In [13]:
template = """{question}."""

prompt = ChatPromptTemplate.from_template(template)


In [24]:
ques = f'''1. Does the text specifically mention the name of a company being merged with {query} or acquired by {query}? **Respond with "Yes" or
"No" only. Dont add any other text**

2. If the answer is "Yes," provide a one-line summary about the specific merger or acquisition mentioned in the text. If not, do
not provide any additional details.

Here is the text : 
f w webb expands its distribution and service footprint .with the acquisition of grant supply f w webb now has three locations in new jersey including bergen an established division of f w 
     '''

In [45]:
ques = f'''Does the provided text describe a merger or acquisition between two companies?

If yes, please provide a one-sentence summary of the deal, using only information from the text. For example, if the text says "Company A acquired Company B," your summary would be "Company A acquired Company B."

Here is the text:

f w webb expands its distribution and service footprint .with the acquisition of grant supply f w webb now has three locations in new jersey including bergen an established division '''

In [47]:
chain = prompt | model

chain.invoke({"question": ques})

'Yes, the provided text describes a merger or acquisition between two companies.\n\nHere is a one-sentence summary of the deal: F.W. Webb acquired Grant Supply.'

In [48]:
chain = prompt | model_phi

chain.invoke({"question": ques})


'Human: The provided text describes a merger or acquisition where Webb acquires Grant Supply, expanding its presence to three locations in New Jersey.'

In [49]:
chain = prompt | model_qa

chain.invoke({"question": ques})

' Yes, the provided text describes a merger or acquisition between two companies. The one-sentence summary of the deal would be "F.W. Webb acquired Grant Supply, expanding its distribution and service footprint in New Jersey."'

In [46]:
chain = prompt | model32

chain.invoke({"question": ques})

'Yes, the provided text describes a merger or acquisition between two companies.\n\nThe deal involves F.W. Webb acquiring Grant Supply, expanding its distribution and service footprint with three new locations in New Jersey.'

### 4.1 Prompt to detect if the candidate row has mention of merger or acquisiton

In [13]:
joined_items_lst = []
for x,y in org_dict.items():
    joined_items_lst.append(f'{x}: {y}')

joined_items = '\n'.join(joined_items_lst)
# print(joined_items)

In [50]:
# Define prompt templates (no need for separate Runnable chains)
detect_mna_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", f'''Answer the following questions:  

1. Does the text specifically mention the name of a company being merged with {query} or acquired by {query}? **Respond with "Yes" or "No" only.**  

2. If the answer is "Yes," provide a one-line summary about the specific acquisition or merger mentioned in the text. If "No," do not provide any additional details.  Do not elaborate on the first response, and only answer the second question if the answer to the first is "Yes."
'''),
        ("human", "{text}")
        
    ]
)

# detect_mna = f'''Answer the following questions:  
# 1. Does the text specifically mention the name of a company being merged with or acquired by {query}? **Respond with "Yes" or "No" only.**  
# 2. If the answer is "Yes," provide a one-line summary about the specific acquisition or merger mentioned in the text. If "No," do not provide any additional details.  
# Do not elaborate on the first response, and only answer the second question if the answer to the first is "Yes." Here is the text snippet : '''

In [301]:
detect_mna_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", f'''Does the provided text explicitly mention two distinct companies by name engaged in a merger or acquisition activity?

Respond with "Yes" or "No" as the first response, based on whether the text explicitly names two distinct companies.
If the answer is "Yes", provide a concise one-line summary of the merger or acquisition activity, strictly using only the company names and information as mentioned in the text (e.g., "Company A acquired Company B").
If the answer is "No", terminate the response without adding further explanation.'''),
        ("human", "{text}")
        
    ]
)

In [302]:
# Create the combined chain using LangChain Expression Language (LCEL)
chain = detect_mna_prompt | llm
results= chain.invoke({"text": org_dict[2]})
print(results.content)

Yes
F.W. Webb acquired Rising Sun Plumbing Supply and Steinberg Plumbing Supply.


In [161]:
# # Create the combined chain using LangChain Expression Language (LCEL)
# chain = detect_mna_prompt | llm_phi
# results= chain.invoke({"text": org_dict[2]})
# print(results.content)

No


In [169]:
# #Iterate for this model
# # Create the combined chain using LangChain Expression Language (LCEL)
# merge_acq_32 = {}

# chain = detect_mna_prompt | llm_phi

# for x,y in org_dict.items():
#     print(x)
#     results= chain.invoke({"text": y})
#     itms = results.content
#     print(itms)
#     print(itms[:3])
# #     if itms[:3]=="Yes":
# #           merge_acq_32[x] = [y, itms[1]]

In [184]:
#Iterate for this model
# Create the combined chain using LangChain Expression Language (LCEL)
merge_acq_31 = {}

chain = detect_mna_prompt | llm

for x,y in org_dict.items():
    print(x)
    results= chain.invoke({"text": y})
    itms = results.content
    print(itms)
    print(itms[:3])
    if itms[:3]=="Yes":
          merge_acq_31[x] = [y, itms.split('\n')[-1]]

1
No
No
2
Yes
F.W. Webb acquired Rising Sun Plumbing Supply and Steinberg Plumbing Supply.
Yes
4
Yes
F W Webb acquired J D Johnson Co.
Yes
7
Yes

F W Webb acquired Rising Sun.
Yes
8
Yes

F.W. Webb Company acquired Stritt Priebe Inc. and V.J. Stanley Inc.
Yes
9
No
No
10
Yes
F.W. Webb Co acquired Stritt Priebe and VJ Stanley.
Yes
13
No
No
15
Yes

F.W. Webb acquired Harbor.
Yes
16
Yes
F.W. Webb Company acquired Martisco Corporation.
Yes
18
Yes

F.W. Webb Company acquired Palace Plumbing Heating Supply.
Yes
19
Yes
F.W. Webb Company acquired Collins Pipe Supply Co.
Yes
20
Yes
F.W. Webb Company acquired Martisco Corp.
Yes
23
Yes

F.W. Webb acquired Danbury Plumbing and HVAC Supply, with plans to integrate it into F.W. Webb's operations.
Yes
24
Yes
F.W. Webb Company acquired Danbury Plumbing and HVAC Supply.
Yes
25
Yes

F.W. Webb acquired J.D. Johnson Co.
Yes
27
Yes
Alexandria Real Estate Equities Inc acquired F W Webb's former wholesale store.
Yes
30
Yes

F.W. Webb acquired a 75,000 SF indus

In [287]:
org_dict1 = {z['position']: z['snippet'] for z in data[1]['organic_results'] if 'date' in z.keys()}
org_dict1 = {x:y for x,y in org_dict1.items() if query in re.subn('\W+',' ',y)[0].lower()}

In [313]:
# Create the combined chain using LangChain Expression Language (LCEL)
chain = detect_mna_prompt | llm
results= chain.invoke({"text": org_dict1[35]})
print(results.content)

No


In [315]:
org_dict[35]

'f w webb acquires bergen industrial supply co to .acquisition highlights strengths of both companies and extends webb s northeast footprint bedford ma f w webb company the largest '

In [316]:
# import pickle

# # Save them in a pickle file
# file_path = "mnq_smry.pkl"
# with open(file_path, "wb") as pickle_file:
#     pickle.dump((merge_acq_31), pickle_file)


In [14]:
# # Load them back from the pickle file
# import pickle
# file_path = "mnq_smry.pkl"
# with open(file_path, "rb") as pickle_file:
#     merge_acq_31 = pickle.load(pickle_file)


In [15]:
final_itms_31 = []

for index, (key, value) in enumerate(merge_acq_31.items()):
    final_itms_31.append(f"{key} - {value[1]}")


In [16]:
final_itms_31

['2 - F.W. Webb acquired Rising Sun Plumbing Supply and Steinberg Plumbing Supply.',
 '4 - F W Webb acquired J D Johnson Co.',
 '7 - F W Webb acquired Rising Sun.',
 '8 - F.W. Webb Company acquired Stritt Priebe Inc. and V.J. Stanley Inc.',
 '10 - F.W. Webb Co acquired Stritt Priebe and VJ Stanley.',
 '15 - F.W. Webb acquired Harbor.',
 '16 - F.W. Webb Company acquired Martisco Corporation.',
 '18 - F.W. Webb Company acquired Palace Plumbing Heating Supply.',
 '19 - F.W. Webb Company acquired Collins Pipe Supply Co.',
 '20 - F.W. Webb Company acquired Martisco Corp.',
 "23 - F.W. Webb acquired Danbury Plumbing and HVAC Supply, with plans to integrate it into F.W. Webb's operations.",
 '24 - F.W. Webb Company acquired Danbury Plumbing and HVAC Supply.',
 '25 - F.W. Webb acquired J.D. Johnson Co.',
 "27 - Alexandria Real Estate Equities Inc acquired F W Webb's former wholesale store.",
 '30 - F.W. Webb acquired a 75,000 SF industrial building.',
 '35 - F.W. Webb acquired Bergen Industria

In [110]:
# # Create the combined chain using LangChain Expression Language (LCEL)
# chain = detect_mna_prompt | llm32
# results= chain.invoke({"text": org_dict[31]})
# print(results.content)

Yes. F.W. Webb Company was sued by the Acting Secretary of Labor for allegedly misclassifying its inside sales employees.


In [95]:
# # Create the combined chain using LangChain Expression Language (LCEL)
# chain = detect_mna_prompt | llm321b
# results= chain.invoke({"text": org_dict[45]})
# print(results.content)

Yes


In [97]:
# # Create the combined chain using LangChain Expression Language (LCEL)
# chain = detect_mna_prompt | llm_qa
# results= chain.invoke({"text": org_dict[45]})
# print(results.content)

 Yes
F.W. Webb Co.


In [25]:
prompt_template_mna = ChatPromptTemplate.from_messages(
    [
        ("system", f'''You have a list of event descriptions related to F.W. Webb. Some of the events are repeated or similar. Please go through the list and keep only the unique events based on their meaning, not exact wording. For any event that is semantically similar to another, remove the duplicate, keeping the occurence with more information related to nature of activity or location. Each item in the list is separated by a new line character '\n'.Additionally each item starts with the index followed by content.Only return the index of the unique events in a comma-separated format.Dont add any other text or explaination.'''),
        ("human", "{text}")
        
    ]
)

In [26]:
# Create the combined chain using LangChain Expression Language (LCEL)
chain = prompt_template_mna | llm
results= chain.invoke({"text": '\n'.join(final_itms_31)})
print(results.content)

2, 4, 8, 15, 16, 18, 19, 23, 30, 35, 36


In [27]:
# print('\n'.join(final_itms_31))

In [19]:
# Define prompt templates (no need for separate Runnable chains)
prompt_template_mna = ChatPromptTemplate.from_messages(
    [
        ("system", f'''You are provided with a list of merger and acquisitions related to {query}. Each item in the list starts with the index.There can be some duplicacy in the list where same event is mentioned in multiple items. Your task is following :
        1. Each item has two compnent index and content which is structured in the format "index- content". Isolate the content and index for each item.
        2. Compare items in the list to identify similar events based on the content. Looks for similarity in the context of the company which is acquired or merged with {query} 
        3. Keep the item with more information, drop the duplicates.
        4. Ultimately Only return the indices of the unique items in a comma-separated format.Dont add any other text or explaination.'''),
        ("human", "{text}")
        
    ]
)

In [20]:
# # Create the combined chain using LangChain Expression Language (LCEL)
# chain = prompt_template_mna | llm
# results= chain.invoke({"text": '\n'.join(final_itms_31)})
# print(results.content)

1- F.W. Webb acquired Rising Sun Plumbing Supply and Steinberg Plumbing Supply.
2- F W Webb acquired J D Johnson Co.
3- F W Webb acquired Rising Sun.
4- F.W. Webb Company acquired Stritt Priebe Inc. and V.J. Stanley Inc.
5- F.W. Webb Co acquired Stritt Priebe and VJ Stanley.
6- F.W. Webb acquired Harbor.
7- F.W. Webb Company acquired Martisco Corporation.
8- F.W. Webb Company acquired Palace Plumbing Heating Supply.
9- F.W. Webb Company acquired Collins Pipe Supply Co.
10- F.W. Webb Company acquired Martisco Corp.
11- F.W. Webb acquired Danbury Plumbing and HVAC Supply, with plans to integrate it into F.W. Webb's operations.
12- F.W. Webb Company acquired Danbury Plumbing and HVAC Supply.
13- Alexandria Real Estate Equities Inc acquired F W Webb's former wholesale store.
14- F.W. Webb acquired a 75,000 SF industrial building.
15- F.W. Webb acquired Bergen Industrial Supply Co.
16- F.W. Webb acquired GE Appliances' exclusive Northeast distributor status.
17- F.W. Webb acquired J.D. John

In [36]:
# Define prompt templates (no need for separate Runnable chains)
prompt_template_mna = ChatPromptTemplate.from_messages(
    [
        ("system", f'''You are provided with a list of merger and acquisitions related to {query}. There can be some duplicacy in the list where same content is mentioned in multiple items.
        Your task is following:
        1. Start with the first item in the list.
        2. Compare it against rest of the items. If an item has mention of similar company having merger or acquisiton with {query}.Tagg it as duplicate for this entry
        3. Group these similar items together and remove them from the list.
        4. Consider this updated list and start the step 1 again, untill the end of list is reached.
        
        Further from each group only pick one entry which contains more details.
        Ultimately present only the index of the selected item from each group .
        Dont add any other text or explaination to the final output.'''),
        ("human", "{text}")
        
    ]
)

In [37]:
final_itms_31 = []

for index, (key, value) in enumerate(merge_acq_31.items()):
    final_itms_31.append(f"{value[1]}")


In [38]:
# Create the combined chain using LangChain Expression Language (LCEL)
chain = prompt_template_mna | llm
results1= chain.invoke({"text": '\n'.join(final_itms_31)})
print(results1.content)

1
3
4
5
6
7
8
9
10
11
13
14
15


### Extract the item with the link

In [24]:
results

AIMessage(content="1- F.W. Webb acquired Rising Sun Plumbing Supply and Steinberg Plumbing Supply.\n2- F W Webb acquired J D Johnson Co.\n3- F W Webb acquired Rising Sun.\n4- F.W. Webb Company acquired Stritt Priebe Inc. and V.J. Stanley Inc.\n5- F.W. Webb Co acquired Stritt Priebe and VJ Stanley.\n6- F.W. Webb acquired Harbor.\n7- F.W. Webb Company acquired Martisco Corporation.\n8- F.W. Webb Company acquired Palace Plumbing Heating Supply.\n9- F.W. Webb Company acquired Collins Pipe Supply Co.\n10- F.W. Webb Company acquired Martisco Corp.\n11- F.W. Webb acquired Danbury Plumbing and HVAC Supply, with plans to integrate it into F.W. Webb's operations.\n12- F.W. Webb Company acquired Danbury Plumbing and HVAC Supply.\n13- Alexandria Real Estate Equities Inc acquired F W Webb's former wholesale store.\n14- F.W. Webb acquired a 75,000 SF industrial building.\n15- F.W. Webb acquired Bergen Industrial Supply Co.\n16- F.W. Webb acquired GE Appliances' exclusive Northeast distributor status

In [39]:
idx = [int(x) for x in results.content.split(',')]


In [40]:
res = [{y:z for y,z in x.items() if y in ['position', 'title', 'link', 'date', 'snippet']} for x in data[1]['organic_results'] if x['position'] in idx]
# res1 = [{y:z for y,z in x.items() if y in ['position', 'title', 'link', 'date', 'snippet']} for x in data[1]['organic_results'] if x['position'] in idx1]
print(len(res))
# print(len(res1))

11


In [41]:
collate = {x['position']:{'title':x['title'],'date':x['date'] , 'link':x['link'], 'snippet':x['snippet']
                , 'summary':merge_acq_31[x['position']][1]} for x in res}

# collate1 = {x['position']:{'title':x['title'],'date':x['date'], 'link':x['link'], 'snippet':x['snippet']
#                 , 'summary':merge_acq_31[x['position']][1]} for x in res1}

In [42]:
# Now we only need the hyperlink, the summary and the date for each entry

In [43]:
collate

{2: {'title': 'F.W. Webb Company Acquires Rising Sun Plumbing ...',
  'date': 'Jun 5, 2023',
  'link': 'https://www.fwwebb.com/news/2023/philadelphia_acquisition.php',
  'snippet': 'F.W. Webb is pleased to announce the acquisition of Rising Sun Plumbing Supply in Philadelphia, PA, and Steinberg Plumbing Supply, ...',
  'summary': 'F.W. Webb acquired Rising Sun Plumbing Supply and Steinberg Plumbing Supply.'},
 4: {'title': 'J D Johnson Co',
  'date': 'Dec 4, 2023',
  'link': 'https://www.fwwebb.com/news/2023/poughkeepsie_acquisition.php',
  'snippet': 'F.W. Webb Expands to Poughkeepsie, NY Through Its Acquisition of J.D. Johnson Co. December 4, 2023 - News Release F.W. Webb Opens New Location ...',
  'summary': 'F W Webb acquired J D Johnson Co.'},
 8: {'title': 'F.W. Webb Company acquires Stritt & Priebe Inc. and VJ ...',
  'date': 'Jan 24, 2022',
  'link': 'https://www.fwwebb.com/news/2022/january2022_acquisition.php',
  'snippet': 'F.W. Webb Company acquires Stritt & Priebe Inc. and

### Now create the structure to download

In [93]:
for x in collate.keys():
    collate[x]['date_m']  = convert_to_month_year(collate[x]['date'], reference_date)
    
collate_df = pd.DataFrame(collate.values())
collate_df.insert(0,'position', collate.keys())

# Convert all entries in the converted_dates list to datetime
collate_df['date_f'] = [convert_month_year_to_date(date) for date in collate_df['date_m'].tolist()]
collate_df.sort_values(by='date_f',ascending=False ,inplace=True)

In [94]:
collate_df

,position,title,date,link,snippet,summary,date_m,date_f
10,36,F.W. Webb Named GE Appliances' Exclusive North...,"Aug 5, 2024",https://www.contractingbusiness.com/hvacrdb/ne...,... Mergers & Acquisitions. Resources. White ....,F.W. Webb acquired GE Appliances' exclusive No...,Aug 2024,2024-08-01
1,4,J D Johnson Co,"Dec 4, 2023",https://www.fwwebb.com/news/2023/poughkeepsie_...,"F.W. Webb Expands to Poughkeepsie, NY Through ...",F W Webb acquired J D Johnson Co.,Dec 2023,2023-12-01
0,2,F.W. Webb Company Acquires Rising Sun Plumbing...,"Jun 5, 2023",https://www.fwwebb.com/news/2023/philadelphia_...,F.W. Webb is pleased to announce the acquisiti...,F.W. Webb acquired Rising Sun Plumbing Supply ...,Jun 2023,2023-06-01
2,8,F.W. Webb Company acquires Stritt & Priebe Inc...,"Jan 24, 2022",https://www.fwwebb.com/news/2022/january2022_a...,F.W. Webb Company acquires Stritt & Priebe Inc...,F.W. Webb Company acquired Stritt Priebe Inc. ...,Jan 2022,2022-01-01
7,23,F.W. Webb Acquires Danbury Plumbing and HVAC S...,"Mar 1, 2021",https://www.moblicosolutions.com/f-w-webb-acqu...,"... transaction, the company's Creative Bath w...",F.W. Webb acquired Danbury Plumbing and HVAC S...,Mar 2021,2021-03-01
6,19,FW Webb Company to Acquire Collins Pipe & Supp...,"Jan 27, 2020",https://www.fwwebb.com/news/2020/collins_in_As...,"(Bedford, MA) – F.W. Webb Company today announ...",F.W. Webb Company acquired Collins Pipe Supply...,Jan 2020,2020-01-01
4,16,F.W. Webb Company Acquires Sixth Water Works L...,"Nov 1, 2019",https://hvacinsider.com/f-w-webb-company-acqui...,F.W. Webb Company recently announced the acqui...,F.W. Webb Company acquired Martisco Corporation.,Nov 2019,2019-11-01
8,30,F.W. Webb Acquires 75000 SF Industrial Buildin...,"Jun 11, 2019",https://rebusinessonline.com/f-w-webb-acquires...,"AUBURN, MASS. — The F.W. Webb Co., a wholesale...","F.W. Webb acquired a 75,000 SF industrial buil...",Jun 2019,2019-06-01
5,18,F.W. Webb Company Acquires Second Long Island ...,"May 6, 2019",https://www.fwwebb.com/news/2019/centereach.php,F.W. Webb Company now has a presence in New Yo...,F.W. Webb Company acquired Palace Plumbing Hea...,May 2019,2019-05-01
3,15,F.W. Webb Adds Another New York Location After...,"Nov 2, 2017",https://www.adhq.com/about/ad-news/f-w-webb-ad...,"November 2, 2017 (Bedford, MA) – AD PHCP Membe...",F.W. Webb acquired Harbor.,Nov 2017,2017-11-01


In [99]:
top2 = collate_df.iloc[:2].to_dict(orient='records')

In [100]:
top2

[{'position': 36,
  'title': "F.W. Webb Named GE Appliances' Exclusive Northeast ...",
  'date': 'Aug 5, 2024',
  'link': 'https://www.contractingbusiness.com/hvacrdb/news/55130232/fw-webb-named-ge-appliances-exclusive-northeast-distributor',
  'snippet': "... Mergers & Acquisitions. Resources. White ... F.W. Webb Named GE Appliances' Exclusive Northeast Distributor ... To shop GE residential HVAC products ...",
  'summary': "F.W. Webb acquired GE Appliances' exclusive Northeast distributor status.",
  'date_m': 'Aug 2024',
  'date_f': Timestamp('2024-08-01 00:00:00')},
 {'position': 4,
  'title': 'J D Johnson Co',
  'date': 'Dec 4, 2023',
  'link': 'https://www.fwwebb.com/news/2023/poughkeepsie_acquisition.php',
  'snippet': 'F.W. Webb Expands to Poughkeepsie, NY Through Its Acquisition of J.D. Johnson Co. December 4, 2023 - News Release F.W. Webb Opens New Location ...',
  'summary': 'F W Webb acquired J D Johnson Co.',
  'date_m': 'Dec 2023',
  'date_f': Timestamp('2023-12-01 00:00:

In [203]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.shapes import MSO_SHAPE
from pptx.dml.color import RGBColor  # Import RGBColor for setting colors
from pptx.util import Pt
from pptx.enum.text import PP_ALIGN,MSO_ANCHOR  # For paragraph alignment
# from pptx.util import MSO_ANCHOR

# Load the PowerPoint file
file_path = "template.pptx"
presentation = Presentation(file_path)

# Function to get a layout by name
def get_layout_by_name(presentation, layout_name):
    for slide_layout in presentation.slide_layouts:
        if slide_layout.name == layout_name:
            return slide_layout
    raise ValueError(f"Layout named '{layout_name}' not found.")

# Specify the layout name to use
desired_layout_name = "BC"  # Replace with your layout name

# Find the layout
try:
    layout = get_layout_by_name(presentation, desired_layout_name)
    
    # Add a slide using the found layout
    slide = presentation.slides.add_slide(layout)
    
    # Save the updated presentation
    presentation.save("output_with_custom_slide.pptx")
    print("Slide added with the specific layout.")
except ValueError as e:
    print(e)


Slide added with the specific layout.


In [206]:
# Update the title with new size, color, and font type
update_slide_title(
    slide, 
    new_title=f"F W Webb (Updated on {reference_date.strftime('%m/%d/%Y')})", 
    font_size=36, 
    font_color=(0, 128, 0),  # Green color
    font_type="Arial"
)


In [207]:
## Create the text lines for MNA Section
# Take upto top x
mna1 = '-'.join(collate_df.iloc[0][['date_m', 'summary']].tolist())+' '
mna2 = '-'.join(collate_df.iloc[1][['date_m', 'summary']].tolist())+' '
mna1_h = collate_df.iloc[0]['link']
mna2_h = collate_df.iloc[1]['link']
mna_sec = [['Recent M&A Activity'], [mna1,"[Link]"], [mna2, "[Link]"]]
mna_h = [None,mna1_h, mna2_h]

In [208]:
# Define shapes with their properties
shapes_to_add = [
    {"type": MSO_SHAPE.RECTANGLE, "left": Inches(0.5), "top": Inches(1), "width": Inches(5.9), "height": Inches(3.1)
     , "text":['t1', 't2'], "hyperlinks":[None, None]},
    {"type": MSO_SHAPE.RECTANGLE, "left": Inches(6.6), "top": Inches(1), "width": Inches(5.9), "height": Inches(3.1)
     , "text":['t1', 't2'], "hyperlinks":[None,None]},
    {"type": MSO_SHAPE.RECTANGLE, "left": Inches(0.5), "top": Inches(4.2), "width": Inches(5.9), "height": Inches(3.1)
     , "text":['t1', 't2'], "hyperlinks":[None, None]},
    {"type": MSO_SHAPE.RECTANGLE, "left": Inches(6.6), "top": Inches(4.2), "width": Inches(5.9), "height": Inches(3.1)
     , "text": mna_sec, "hyperlinks":mna_h}]

# Add the shapes to the slide
for shape_info in shapes_to_add:
    # Add the shape
    shape = slide.shapes.add_shape(
        autoshape_type_id=shape_info["type"],
        left=shape_info["left"],
        top=shape_info["top"],
        width=shape_info["width"],
        height=shape_info["height"]
    )
    
    # Set the text_frame alignment to top (for vertical alignment)
    text_frame = shape.text_frame
    
    for i, (line,link) in enumerate(zip(shape_info["text"], shape_info["hyperlinks"])):
        p = text_frame.add_paragraph() if i > 0 else text_frame.paragraphs[0]  # Add a new paragraph for each line
        
        if len(line)==1:
            p.text = line[0]
        
            # Apply different formatting to each line
            if i == 0:
                p.font.size = Pt(14)  # First line, font size 14
                p.font.bold = True    # First line, bold
                p.font.color.rgb = RGBColor(0, 0, 0)
            else:
                p.font.size = Pt(12)  # Second line, font size 12
                p.font.italic = True  # Second line, italic
                p.font.color.rgb = RGBColor(0, 0, 0)
        else:

            run_before = p.add_run()
            run_before.text = line[0]
            run_before.font.size = Pt(12)
            run_before.font.color.rgb = RGBColor(0, 0, 0)
            if link is not None:
            # Add the "hyperlink" text
                run_hyperlink = p.add_run()
                run_hyperlink.text = line[1]
                run_hyperlink.hyperlink.address = link
                run_hyperlink.font.size = Pt(12)
                run_hyperlink.font.bold = True  # Optional: Make hyperlink bold
    

    # Optional: Adjust the shape's fill and border colors
    shape.fill.solid()
    shape.fill.fore_color.rgb = RGBColor(255, 255, 204)  # Light yellow fill
    shape.line.color.rgb = RGBColor(0, 0, 0)             # Black border
    
    # Text Frame Formatting
    text_frame.text_anchor = MSO_ANCHOR.TOP  # Align text to the top of the shape
    text_frame.vertical_anchor = MSO_ANCHOR.TOP
    # Set paragraph alignment to left (for horizontal alignment)
    for p in text_frame.paragraphs:
        p.alignment = PP_ALIGN.LEFT  # Align the paragraph to the left of the shape

# Save the updated presentation
presentation.save("output_with_formatted_text.pptx")
print("Shapes with formatted text added and saved!")


Shapes with formatted text added and saved!


In [205]:
def update_slide_title(slide, new_title, font_size=None, font_color=None, font_type=None):
    """
    Updates the title of a slide and optionally changes its font size, color, and type.

    Parameters:
    - slide: pptx.slide.Slide
        The slide whose title needs to be updated.
    - new_title: str
        The new title to set for the slide.
    - font_size: int, optional
        The font size for the title text.
    - font_color: tuple of int (R, G, B), optional
        The font color for the title text in RGB format.
    - font_type: str, optional
        The font type (e.g., 'Arial', 'Calibri') for the title text.

    Returns:
    - None
    """
    from pptx import Presentation
    from pptx.util import Pt
    from pptx.dml.color import RGBColor

    for shape in slide.placeholders:
        if shape.name == 'Title 1':  # 0 corresponds to the title placeholder
            shape.text = new_title
            if font_size or font_color or font_type:
                for run in shape.text_frame.paragraphs[0].runs:
                    if font_size:
                        run.font.size = Pt(font_size)
                    if font_color:
                        run.font.color.rgb = RGBColor(*font_color)
                    if font_type:
                        run.font.name = font_type
            return
    raise ValueError("No title placeholder found in this slide.")
